In [1]:
import os
import MDAnalysis as mda
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
u = mda.Universe('../start_fep.pdb')

ag_lig_1 = u.select_atoms('resname EPJ').split('residue')[0]

prot_binding_site = u.select_atoms('name CA and byres (protein and around 4 group ag_lig_1)', ag_lig_1=ag_lig_1)

#sort prot_resids
chain_resids = prot_binding_site.segids[np.argsort(prot_binding_site.resids)]
prot_resids = np.sort(prot_binding_site.resids)

prot_resids = prot_resids[np.argsort(chain_resids)]
chain_resids = np.sort(chain_resids)

print(prot_resids)
print(chain_resids)

next_subunit_dict = {'A': 'C',
                     'B': 'D',
                     'C': 'E',
                     'D': 'F',
                     'E': 'G',
                     'F': 'H',
                     'G': 'I',
                     'H': 'J',
                     'I': 'A',
                     'J': 'B'}

prot_selections = []

for i in range(5):
    # only makes senses for homomers
    selection = 'protein and ('
    for j in range(prot_resids.shape[0]):
        # iterate subunit
        selection += f'(resid {prot_resids[j]} and segid {chain_resids[j]}) or '
        chain_resids[j] = next_subunit_dict[chain_resids[j]]
    selection = selection[:-4] + ')'
    prot_selections.append(selection)

for i, selection in enumerate(prot_selections):
    print(selection)
    ag = u.select_atoms(selection)
    ag.write(f'bining_sites.vmd', mode='a',name='mda_' + str(i))

# see: https://docs.mdanalysis.org/stable/documentation_pages/selections/vmd.html#MDAnalysis.selections.vmd.SelectionWriter

[100 155 156 157 197 199 200 204  57  81 109 110 111 119 121]
['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'C' 'C' 'C']
protein and ((resid 100 and segid A) or (resid 155 and segid A) or (resid 156 and segid A) or (resid 157 and segid A) or (resid 197 and segid A) or (resid 199 and segid A) or (resid 200 and segid A) or (resid 204 and segid A) or (resid 57 and segid C) or (resid 81 and segid C) or (resid 109 and segid C) or (resid 110 and segid C) or (resid 111 and segid C) or (resid 119 and segid C) or (resid 121 and segid C))
protein and ((resid 100 and segid C) or (resid 155 and segid C) or (resid 156 and segid C) or (resid 157 and segid C) or (resid 197 and segid C) or (resid 199 and segid C) or (resid 200 and segid C) or (resid 204 and segid C) or (resid 57 and segid E) or (resid 81 and segid E) or (resid 109 and segid E) or (resid 110 and segid E) or (resid 111 and segid E) or (resid 119 and segid E) or (resid 121 and segid E))
protein and ((resid 100 and segid E) or (resid 1

# Binding site restraints

In [4]:
binding_sites = []

# only two binding sites
for selection in np.asarray(prot_selections)[[0,3]]:
    binding_sites.append(u.select_atoms(selection))

with mda.selections.gromacs.SelectionWriter('../index_fep.ndx', mode='w') as ndx:
    ndx.write(u.atoms, name='SYSTEM')
    ndx.write(u.select_atoms('protein'), name='PROT')
    ndx.write(u.select_atoms('resname POPC'), name='MEMB')
    ndx.write(u.select_atoms('resname TIP3 SOD CLA EPJ'), name='SOL_ION')
    ndx.write(u.select_atoms('resname EPJ'), name='EPJ')
    for i, epj_agg in enumerate(u.select_atoms('resname EPJ').split('residue')):
        ndx.write(epj_agg, name=f'EPJ{i+1}')
        ndx.write(epj_agg.select_atoms('name N1'), name=f'EPJ{i+1}_NT')
        ndx.write(epj_agg.select_atoms('name C10'), name=f'EPJ{i+1}_CT')
    for i, binding_site in enumerate(binding_sites):
        ndx.write(binding_site, name=f'BS{i+1}')

# Loop C

In [5]:
#LoopC resids: chain A G and resid 193-206
#LoopC resids: chain C E I and resid 185-198

In [6]:
ch_1_loopc = u.select_atoms('segid A and resid 193-206 and name CA')
ch_2_loopc = u.select_atoms('segid C and resid 185-198 and name CA')
ch_3_loopc = u.select_atoms('segid E and resid 185-198 and name CA')
ch_4_loopc = u.select_atoms('segid G and resid 193-206 and name CA')
ch_5_loopc = u.select_atoms('segid I and resid 185-198 and name CA')

loopc_ags = [ch_1_loopc, ch_2_loopc, ch_3_loopc, ch_4_loopc, ch_5_loopc]

In [7]:
with mda.selections.gromacs.SelectionWriter('../index_fep.ndx', mode='a') as ndx:
    for i, loopc in enumerate(loopc_ags):
        ndx.write(loopc, name=f'LoopC{i+1}')